In [28]:
#import libaries
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO
from google.cloud import storage

In [29]:
#extract data function
def extract_data_dolthub(repo, branch, table):
    url = f"https://www.dolthub.com/csv/dolthub/{repo}/{branch}/{table}"
    df_raw = pd.read_csv(url)
    print(df_raw.info())
    print(df_raw.shape)
    return df_raw

In [31]:
#function to store data to the google cloud
def store_to_google_bucket(df):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket("different-state-hospital-price")
    blob = bucket.blob(f"{df}.json")
    df.to_json(f"{df}.json")
    blob.upload_from_filename(f"{df}.json")

In [33]:
df_raw = extract_data_dolthub('us-jails', 'main', 'incidents')
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6721 entries, 0 to 6720
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         6721 non-null   int64  
 1   start_date                 6721 non-null   object 
 2   end_date                   6721 non-null   object 
 3   deaths                     6721 non-null   int64  
 4   fights                     0 non-null      float64
 5   total_assaults             0 non-null      float64
 6   inmate_on_inmate_assaults  0 non-null      float64
 7   inmate_on_staff_assaults   0 non-null      float64
 8   suicides                   6721 non-null   int64  
 9   escapes                    0 non-null      float64
 10  source_url                 6721 non-null   object 
 11  source_url_2               6721 non-null   object 
dtypes: float64(5), int64(3), object(4)
memory usage: 630.2+ KB
None
(6721, 12)


,id,start_date,end_date,deaths,fights,total_assaults,inmate_on_inmate_assaults,inmate_on_staff_assaults,suicides,escapes,source_url,source_url_2
0,10172,2013-02-24,2013-02-25,1,NaN,NaN,NaN,NaN,0,NaN,https://www.reuters.com/investigates/special-r...,https://graphics.thomsonreuters.com/data/jails...
1,10172,2013-03-03,2013-03-04,1,NaN,NaN,NaN,NaN,0,NaN,https://www.reuters.com/investigates/special-r...,https://graphics.thomsonreuters.com/data/jails...
2,10172,2013-07-29,2013-07-30,1,NaN,NaN,NaN,NaN,0,NaN,https://www.reuters.com/investigates/special-r...,https://graphics.thomsonreuters.com/data/jails...
3,10172,2014-05-25,2014-05-26,1,NaN,NaN,NaN,NaN,1,NaN,https://www.reuters.com/investigates/special-r...,https://graphics.thomsonreuters.com/data/jails...
4,10172,2014-10-07,2014-10-08,1,NaN,NaN,NaN,NaN,1,NaN,https://www.reuters.com/investigates/special-r...,https://graphics.thomsonreuters.com/data/jails...


In [34]:
store_to_google_bucket(df_raw)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.